In [ ]:
import pandas as pd
anno = pd.read_csv("/Users/oo/Desktop/multi_omics_data/anno.csv", usecols=['Name', 'Relation_to_Island', 'UCSC_RefGene_Name', 'UCSC_RefGene_Group'])
new_anno = anno[anno['Relation_to_Island'] == 'Island'].dropna(axis=0, how='any').reset_index(drop=True)
tss_region = new_anno[new_anno['UCSC_RefGene_Group'].str.contains("TSS1500|TSS200",na=False)].reset_index(drop=True)
tss_region = tss_region.rename(columns={"Name": "ProbeID"})
tss_region = tss_region[['ProbeID', 'UCSC_RefGene_Name', 'UCSC_RefGene_Group']]
tss_region
def combine(r):
    gene_name = r['UCSC_RefGene_Name'].split(';')
    gene_group = r['UCSC_RefGene_Group'].split(';')
    kinds = set()
    for i in range(len(gene_name)):
        if gene_group[i] == 'TSS1500' or gene_group[i] == 'TSS200':
            kinds.add(gene_name[i])

    return list(kinds)

tss_region['combined'] = tss_region.apply(combine, axis=1)
filtered_tss_region = pd.DataFrame(tss_region.explode("combined", ignore_index=True).groupby("combined")['ProbeID'].apply(list)).reset_index(names="gene")

# filtered_tss_region

In [ ]:
import pandas as pd
DNAmm_df = pd.read_csv("/Users/oo/Desktop/1208/independent testing data/final_DNAm_df.csv")
clinical_df = DNAmm_df.iloc[:, :4]
clinical_df = clinical_df.drop(columns='Unnamed: 0')
DNAmm_df = DNAmm_df.iloc[:, 4:]

In [ ]:
# DNAmm_df = pd.read_csv("/Users/oo/Desktop/multi_omics_data/DNAmm.csv")
unique_genes = filtered_tss_region['gene'].unique()
final_DNAm = pd.DataFrame(columns=filtered_tss_region['gene'], index=DNAmm_df.index)
for gene in unique_genes:
    probe_ids = filtered_tss_region[filtered_tss_region['gene'] == gene]['ProbeID'].values[0]
#     print(probe_ids)
#     print(DNAmm_df.loc[:, DNAmm_df.columns.isin(['cg02230017'])].values)
#     print(DNAmm_df.loc[:, DNAmm_df.columns.isin(['cg00008713'])].values)
    all_values = DNAmm_df.loc[:, DNAmm_df.columns.isin(probe_ids)].values
    if all_values.size > 0:
        final_DNAm[gene] = DNAmm_df.loc[:, DNAmm_df.columns.isin(probe_ids)].mean(axis=1)
    
    
DNAm_gene_level = final_DNAm.dropna(axis=1)
DNAm_gene_level
# DNAm_gene_level.to_csv("/Users/oo/Desktop/1208/independent testing data/final_DNAm_df.csv")

In [ ]:
DNAm_gene_level
final_DNAm_df = pd.concat([clinical_df, DNAm_gene_level], axis=1)
final_DNAm_df.to_csv("/Users/oo/Desktop/1208/independent testing data/final_DNAm_df.csv")